# Using [`xarray_data_accessor`](https://github.com/LimnoTech/Xarray-DataAccessor) to rapidly prepare GSSHA input files

**Author:** [Xavier Nogueira](https://github.com/xaviernogueira)

**Problem Statement:** The [Gridded Surface Subsurface Hydrologic Analysis (GSSHA)](https://www.xmswiki.com/wiki/GSSHA) model requires a number of input files to run a simulation. Collecting these inputs, for example Hydro-Meterological (HMET) Data for a given watershed can be time consuming. Additionally, converting raw data into GSSHA formatted ASCII input files via WMES requires saving large data files locally. This notebook demonstrates how to use the [`xarray_data_accessor`](https://github.com/LimnoTech/Xarray-DataAccessor), an easily installable Python package, to rapidly prepare GSSHA input files using ERA5 data.

In [3]:
# import dependencies
import shapely
import typing
import metpy
import dataclasses
import geopandas as gpd
import xarray as xr
import numpy as np
import hvplot.xarray
import hvplot.pandas
import cartopy.crs as ccrs
from pathlib import Path

# import our library
import xarray_data_accessor

# 👨‍💻 Explore `xarray_data_accessor` Offerings 👩‍💻

### Assess available data

In [4]:
# see installed data accessors
xarray_data_accessor.DataAccessorFactory.data_accessor_names()

['AWSDataAccessor', 'CDSDataAccessor', 'NASA_LPDAAC_Accessor']

In [5]:
# see ERA5 variables available from AWS
xarray_data_accessor.DataAccessorFactory.supported_datasets()

{'AWSDataAccessor': ['reanalysis-era5-single-levels'],
 'CDSDataAccessor': ['reanalysis-era5-single-levels',
  'reanalysis-era5-single-levels-preliminary-back-extension',
  'reanalysis-era5-land'],
 'NASA_LPDAAC_Accessor': ['NASADEM_NC', 'NASADEM_SC', 'GLanCE30']}

In [6]:
# see ERA5 variables available from AWS
xarray_data_accessor.DataAccessorFactory.supported_variables(
     'AWSDataAccessor',
    dataset_name='reanalysis-era5-single-levels',
)

['eastward_wind_at_10_metres',
 'northward_wind_at_10_metres',
 'eastward_wind_at_100_metres',
 'northward_wind_at_100_metres',
 'dew_point_temperature_at_2_metres',
 'air_temperature_at_2_metres',
 'air_temperature_at_2_metres_1hour_Maximum',
 'air_temperature_at_2_metres_1hour_Minimum',
 'air_pressure_at_mean_sea_level',
 'sea_surface_wave_mean_period',
 'sea_surface_wave_from_direction',
 'significant_height_of_wind_and_swell_waves',
 'snow_density',
 'lwe_thickness_of_surface_snow_amount',
 'surface_air_pressure',
 'integral_wrt_time_of_surface_direct_downwelling_shortwave_flux_in_air_1hour_Accumulation',
 'precipitation_amount_1hour_Accumulation']

### Assess available data conversion functions

Data conversion functions are stored in classes that can be imported from the `xarray_data_accessor.data_converters` module.

The following data conversion classes are available:
* `ConvertToTable`: Stores functions for converting xarray datasets in to a pandas table format.
* `ConvertToGSSHA`: Stores functions for converting xarray datasets in to GSSHA input files.

`ConvertToGSSHA` contains the following functions:
* `make_gssha_precipitation_input()`
* `make_gssha_grass_ascii()`
* `make_gssha_hmet_wes()`


# 🏞️ Define Our Watershed of Interest 🏞️

Note that above we can see that our GSSHA functions come from a single "plugin" class (`ConvertToGSSHA`).

In [7]:
DATA_DIR = Path.cwd() / 'example_data'

In [8]:
watershed = gpd.GeoDataFrame.from_file(DATA_DIR / 'inflitration_export_poly.shp')
point = watershed.geometry.centroid

In [9]:
# plot our basin(s) and point
watershed.hvplot(
    crs=watershed.crs.to_wkt(),
    tiles='StamenTerrainRetina',
    width=500,
    height=500,
    fill_color=None,
    line_width=4,
    line_color='blue',
)

c:\Users\xrnogueira\Miniconda3\envs\data_accessor_full\Lib\site-packages\osgeo\osr.py:385: FutureWarning: Neither osr.UseExceptions() nor osr.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


:Overlay
   .Tiles.I    :Tiles   [x,y]
   .Polygons.I :Polygons   [Longitude,Latitude]

# 🌧️ Make Precipitation Input File 🌧️

**Method:**
1. Use `xarray_data_accessor.get_xarray_dataset()` to access hourly accumulated precipitation from Planet OS's AWS ERA5 Cloud bucket.
2. Convert units to mm.
3. Generate a precipitation input file with cell centroids as "gages" using `make_gssha_precipitation_input()`.

See [GSSHA Documentation](https://www.gsshawiki.com/Precipitation:Spatially_and_Temporally_Varied_Precipitation).

**Note:** Our GSSHA Model is in NAD83, UTM Zone 15 Projection [EPSG:26915](https://epsg.io/26915).

## 🛰️ Read-In Precipitation Data 🛰️

In [10]:
%%time
precip_data = xarray_data_accessor.get_xarray_dataset(
    data_accessor_name='CDSDataAccessor',
    dataset_name='reanalysis-era5-single-levels',
    variables=['total_precipitation'],
    start_time='2023-03-01T00:00:00',
    end_time='2023-03-31T23:00:00',
    shapefile=watershed,
)

CPU times: total: 4.14 s
Wall time: 1min 1s


In [11]:
precip_data

<xarray.Dataset>
Dimensions:              (longitude: 1, latitude: 1, time: 744)
Coordinates:
  * longitude            (longitude) float32 -90.01
  * latitude             (latitude) float32 38.73
  * time                 (time) datetime64[ns] 2023-03-01 ... 2023-03-31T23:0...
    spatial_ref          int32 0
Data variables:
    total_precipitation  (time, latitude, longitude) float32 0.0 ... 0.0001938
Attributes:
    dataset_name:  reanalysis-era5-single-levels
    institution:   ECMWF
    x_dim:         longitude
    y_dim:         latitude
    EPSG:          4326
    time_step:     hourly
    time_zone:     UTC

## ✍️ Write the ASCII File ✍️

In [12]:
# multiply by 1000 to go from meters to mm
precip_data = precip_data * 1000

In [13]:
%%time
out_file = xarray_data_accessor.ConvertToGSSHA.make_gssha_precipitation_input(
    xarray_dataset=precip_data,
    precipitation_variable='total_precipitation',
    precipitation_type='ACCUM',
    file_name='march_precipitation',
    output_epsg=26915,
)
out_file

CPU times: total: 500 ms
Wall time: 658 ms


WindowsPath('c:/Users/xrnogueira/Documents/Xarray-DataAccessor/examples/march_precipitation.gag')

In [14]:
# clean out temporary files
xarray_data_accessor.delete_temp_files()
del precip_data

# ☀️ Make HMET Input File(s) ☀️

GSSHA Requires Hydro-Meteorological (HMET) data to run a continous simulation. There are multiple input formats possibe, but (so far) out `ConvertToGSSHA` data conversion plugin supports only the two "recommended formats":
* HMET_WES (via `make_gssha_hmet_wes()`) - Creates an ASCII file with singular for the watershed value per time step across 7 HMET variables.
* HMET_ASCII Gridded Data (via `make_gssha_grass_ascii()`) - Creates a single GRASS ASCII grid file for each timestep/variable combinations.

See [GSSHA Documentation](https://www.gsshawiki.com/Continuous:Hydrometeorological_Data).

## 📋 See required HMET variables 📋

All the required HMET variables, their units, nodata values, and alternative names are stored in the `xarray_data_accessor.info.gssha` module's `HMETVariables` dictionary.

In [15]:
for key, value in xarray_data_accessor.info.gssha.HMETVariables.items():
    print(f'{key} | {value.units}')

Barometric Pressure | in Hg
Relative Humidity | %
Total Sky Cover | %
Wind Speed | kts
Dry Bulb Temperature | F
Direct Radiation | W*h/m^2
Global Radiation | W*h/m^2


## 🛰️ Read-In HMET Data from ERA5 🛰️

**Note:** ERA5 uses a different units and different variable structure from GSSHA. Therefore some post-processing is necessary.

In [16]:
%%time
hmet_data = xarray_data_accessor.get_xarray_dataset(
    data_accessor_name='CDSDataAccessor',
    dataset_name='reanalysis-era5-single-levels',
    variables=[
        'surface_pressure',
        '2m_dewpoint_temperature', # to calculate relative humidity
        '2m_temperature',
        'total_cloud_cover',
        '10m_u_component_of_wind', # used to calculate wind speed
        '10m_v_component_of_wind', # used to calculate wind speed
        'total_sky_direct_solar_radiation_at_surface',
        'surface_solar_radiation_downwards',
    ],
    start_time='2023-03-01T00:00:00',
    end_time='2023-03-31T23:00:00',
    shapefile=watershed,
)

CPU times: total: 9.06 s
Wall time: 2min 31s


In [17]:
hmet_data

<xarray.Dataset>
Dimensions:                            (longitude: 1, latitude: 1, time: 744)
Coordinates:
  * longitude                          (longitude) float32 -90.01
  * latitude                           (latitude) float32 38.73
  * time                               (time) datetime64[ns] 2023-03-01 ... 2...
    spatial_ref                        int32 0
Data variables:
    surface_pressure                   (time, latitude, longitude) float32 9....
    2m_dewpoint_temperature            (time, latitude, longitude) float32 27...
    2m_temperature                     (time, latitude, longitude) float32 28...
    total_cloud_cover                  (time, latitude, longitude) float32 0....
    10m_u_component_of_wind            (time, latitude, longitude) float32 -3...
    10m_v_component_of_wind            (time, latitude, longitude) float32 1....
    surface_solar_radiation_downwards  (time, latitude, longitude) float32 1....
    surface_net_solar_radiation        (time, latitude, longitude) float32 1....
Attributes:
    dataset_name:  reanalysis-era5-single-levels
    institution:   ECMWF
    x_dim:         longitude
    y_dim:         latitude
    EPSG:          4326
    time_step:     hourly
    time_zone:     UTC

## 🧮 Make Conversions and Calculations 🧮

In [18]:
import metpy.calc
from metpy.units import units

calculated_rel_humidity = False
calculated_wind_speed = False
converted_clouds = False
converted_temp = False
converted_pressure = False
converted_direct_radiation = False
converted_global_radiation = False

### Calculate Relative Humidity % 

In [19]:
%%time
if not calculated_rel_humidity:
    relative_humidity = metpy.calc.relative_humidity_from_dewpoint(
        temperature=(hmet_data['2m_temperature'] * units.degK),
        dewpoint =(hmet_data['2m_dewpoint_temperature'] * units.degK),
    ).metpy.convert_units(units.percent)

    # convert to %
    hmet_data['relative_humidity'] = relative_humidity.metpy.dequantify().transpose(*hmet_data.dims)
    hmet_data['relative_humidity'].attrs['units'] = '%'
    
    # save memory
    del relative_humidity
    hmet_data = hmet_data.drop_vars('2m_dewpoint_temperature')
    calculated_rel_humidity = True

CPU times: total: 31.2 ms
Wall time: 29.1 ms


### Calculate wind speed in kts

In [20]:
if not calculated_wind_speed:
    wind_speed = metpy.calc.wind_speed(
        (hmet_data['10m_u_component_of_wind'] * units.meter_per_second),
        (hmet_data['10m_v_component_of_wind'] * units.meter_per_second),
    ).metpy.convert_units(units.kts)

    # convert to kts
    hmet_data['wind_speed'] = wind_speed.metpy.dequantify().transpose(*hmet_data.dims)
    hmet_data['wind_speed'].attrs['units'] =  'kts'
    
    # save memory
    del wind_speed
    hmet_data = hmet_data.drop_vars(
        ['10m_u_component_of_wind', '10m_v_component_of_wind'],
    )
    calculated_wind_speed = True

### Convert cloud cover to a % 

In [21]:
if not converted_clouds:
    hmet_data['total_cloud_cover'] = hmet_data['total_cloud_cover'] * 100
    hmet_data['total_cloud_cover'].attrs['units'] = '%'
    converted_clouds = True

### Convert Temperature to F

In [22]:
if not converted_temp:
    hmet_data['2m_temperature'] =  1.8 * (hmet_data['2m_temperature'] - 273) + 32
    hmet_data['2m_temperature'].attrs['units'] = 'F'
    converted_temp = True

### Convert atmospheric pressure to in*Hg

In [23]:
if not converted_pressure:
    hmet_data['surface_pressure'] = (hmet_data['surface_pressure'] * units.Pa).metpy.convert_units(
        units.inHg
    ).metpy.dequantify()
    hmet_data['surface_pressure'].attrs['units'] = 'in Hg'
    converted_temp = True

### Convert both direct and global radition to W/m^2

In [ ]:
if not converted_direct_radiation:
    hmet_data['total_sky_direct_solar_radiation_at_surface'] =  hmet_data['total_sky_direct_solar_radiation_at_surface'] / 3600
    hmet_data['total_sky_direct_solar_radiation_at_surface'].attrs['units'] = 'W/m^2'
    converted_direct_radiation = True

if not converted_global_radiation:
    hmet_data['surface_solar_radiation_downwards'] =  hmet_data['surface_solar_radiation_downwards'] / 3600
    hmet_data['surface_solar_radiation_downwards'].attrs['units'] = 'W/m^2'
    converted_global_radiation = True

In [24]:
hmet_data

<xarray.Dataset>
Dimensions:                            (longitude: 1, latitude: 1, time: 744)
Coordinates:
  * longitude                          (longitude) float32 -90.01
  * latitude                           (latitude) float32 38.73
  * time                               (time) datetime64[ns] 2023-03-01 ... 2...
    spatial_ref                        int32 0
Data variables:
    surface_pressure                   (time, latitude, longitude) float32 29...
    2m_temperature                     (time, latitude, longitude) float32 57...
    total_cloud_cover                  (time, latitude, longitude) float32 0....
    surface_solar_radiation_downwards  (time, latitude, longitude) float32 1....
    surface_net_solar_radiation        (time, latitude, longitude) float32 1....
    relative_humidity                  (longitude, latitude, time) float32 53...
    wind_speed                         (longitude, latitude, time) float32 10...
Attributes:
    dataset_name:  reanalysis-era5-single-levels
    institution:   ECMWF
    x_dim:         longitude
    y_dim:         latitude
    EPSG:          4326
    time_step:     hourly
    time_zone:     UTC

## ✍️ Write HMET_WES Format Input File ✍️

In [25]:
hmet_data.data_vars

Data variables:
    surface_pressure                   (time, latitude, longitude) float32 29...
    2m_temperature                     (time, latitude, longitude) float32 57...
    total_cloud_cover                  (time, latitude, longitude) float32 0....
    surface_solar_radiation_downwards  (time, latitude, longitude) float32 1....
    surface_net_solar_radiation        (time, latitude, longitude) float32 1....
    relative_humidity                  (longitude, latitude, time) float32 53...
    wind_speed                         (longitude, latitude, time) float32 10...

In [26]:
TO_GSSHA_CROSSWALK = {
    'surface_pressure': 'Barometric Pressure',
    'relative_humidity': 'Relative Humidity',
    'total_cloud_cover': 'Total Sky Cover',
    'wind_speed': 'Wind Speed',
    '2m_temperature': 'Dry Bulb Temperature',
    'total_sky_direct_solar_radiation_at_surface': 'Direct Radiation',
    'surface_solar_radiation_downwards': 'Global Radiation',
}

In [27]:
%%time
out_file = xarray_data_accessor.ConvertToGSSHA.make_gssha_hmet_wes(
    hmet_data,
    variable_to_hmet=TO_GSSHA_CROSSWALK,
    file_name='ERA5_HMET_inputs',
)
out_file

CPU times: total: 156 ms
Wall time: 206 ms


WindowsPath('c:/Users/xrnogueira/Documents/Xarray-DataAccessor/examples/ERA5_HMET_inputs.asc')

## ✍️ Write GRASS ASCII Format HMET Inputs ✍️

**Note:** This creates many files, therefore we will only be demo-ing with one variable and a shortened time range.

In [28]:
%%time
out_file = xarray_data_accessor.ConvertToGSSHA.make_gssha_grass_ascii(
    hmet_data,
    variable=list(TO_GSSHA_CROSSWALK.keys())[0],
    hmet_variable=list(TO_GSSHA_CROSSWALK.values())[0],
    end_time='2023-03-01T03:00:00',
    output_epsg=26915,
)
out_file

CPU times: total: 172 ms
Wall time: 186 ms


[WindowsPath('c:/Users/xrnogueira/Documents/Xarray-DataAccessor/examples/2023030100_Pres.asc'),
 WindowsPath('c:/Users/xrnogueira/Documents/Xarray-DataAccessor/examples/2023030101_Pres.asc'),
 WindowsPath('c:/Users/xrnogueira/Documents/Xarray-DataAccessor/examples/2023030102_Pres.asc'),
 WindowsPath('c:/Users/xrnogueira/Documents/Xarray-DataAccessor/examples/2023030103_Pres.asc')]

In [29]:
xarray_data_accessor.delete_temp_files()